In [1]:
import math
import numpy as np
from scipy.linalg import logm
import os
import sys
from abc import abstractmethod
from typing import TypeVar

import argparse
from open3d import *
from pose_utils import gen_coarse_pose_rand
from algs import ICP
from utils import CalPoseDist
import time
sys.path.append("../ffb6d")
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import CyclicLR
import torch.backends.cudnn as cudnn

from datasets import Dataset

from ope_utils import mesh_utils
from configs import config
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
cfg_file = "configs/linemod_ape.yaml"
cfg = config.create_config(cfg_file)

In [2]:
class Params:
    #network parameterws
    def __init__(self):
        self.max_iters = 100
        self.threshold = 1e-5
        self.save_poses = False
        self.outlier = 0.01


params = Params()

In [3]:
cudnn.benchmark = False
cudnn.deterministic = True
torch.manual_seed(0)
torch.set_printoptions(precision=10)
torch.cuda.set_device(0)
torch.manual_seed(0)
train_ds = Dataset(cfg,cfg.train_data)
print(train_ds.cfg.dataset.data_dir)
model_path = os.path.join(train_ds.cfg.dataset.data_dir,"models",f'obj_{train_ds.cls_id:02d}.ply')
model_xyz = torch.tensor(mesh_utils.get_p3ds_from_ply(model_path))/100
print(model_xyz.shape)
train_loader = torch.utils.data.DataLoader(
            train_ds, batch_size=1, shuffle=False,
            drop_last=True, num_workers=4, pin_memory=True)

cls_id in dataset.py 1
train.txt_dataset_size:  186
/home/pxu/ope_ffb6d/ffb6d/datasets/linemod/Linemod_preprocessed
loading p3ds from ply: /home/pxu/ope_ffb6d/ffb6d/datasets/linemod/Linemod_preprocessed/models/obj_01.ply
finish loading ply.
torch.Size([5841, 3])


In [4]:
icp_refiner = ICP(params.max_iters,params.threshold)
errors = {'rot':[],'trans':[]}
accuracy = {'rot':[],'trans':[]}
total_time = 0.0
count = 0
src_pcd = geometry.PointCloud()
dst_pcd = geometry.PointCloud()
src_pcd.points = utility.Vector3dVector(np.array(model_xyz.squeeze()))
data = list(train_loader)[0]

In [10]:
gt_pose = data['RTs']
pt_cloud = data['cld_xyz0']
gt_pose = gt_pose[0,0,...]
init_pose,errs = gen_coarse_pose_rand(gt_pose)
dst_pcd.points = utility.Vector3dVector(np.array(pt_cloud.squeeze()))
start = time.time()
ref_pose,iter = icp_refiner(model_xyz,pt_cloud,init_pose)
#trans_init = np.eye(4)
#trans_init[:3,:4] = init_pose.squeeze()
#res =  registration.registration_icp(src_pcd, dst_pcd, params.threshold, trans_init,registration.TransformationEstimationPointToPoint())
inf_time = time.time()-start
#ref_pose = res.transformation[:3,:]
init_rot,init_trans = CalPoseDist(gt_pose,init_pose.squeeze())
rot_err,trans_err = CalPoseDist(gt_pose,ref_pose)
#print(init_rot,init_trans)
#print(rot_err,trans_err)
errors['rot'].append(init_rot)
errors['trans'].append(init_trans)
accuracy['rot'].append(rot_err)
accuracy['trans'].append(trans_err)
total_time += inf_time
count+=1

In [5]:
print(f"generate error")
print (f"Rot: {np.mean(errors['rot'])}; Trans: {np.mean(errors['trans'])}")
print (f"Rot: {np.std(errors['rot'])}; Trans: {np.std(errors['trans'])}")
print(f"Refined error")
print (f"Rot: {np.mean(accuracy['rot'])}; Trans: {np.mean(accuracy['trans'])}")
print(f"Refined Standard Deviation")
print (f"Rot: {np.std(accuracy['rot'])}; Trans: {np.std(accuracy['trans'])}")
print(f"Runtime:{total_time/count}")

generate error
Rot: nan; Trans: nan
Rot: nan; Trans: nan
Refined error
Rot: nan; Trans: nan
Refined Standard Deviation
Rot: nan; Trans: nan


ZeroDivisionError: float division by zero

In [15]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    visualization.draw_geometries([source_temp, target_temp])
    return visualizer

In [13]:
import copy

In [17]:

pose_vis = np.eye(4)
pose_vis[:3,:] = ref_pose
print(ref_pose)
visualizer = draw_registration_result(src_pcd, dst_pcd, pose_vis)


tensor([[ 0.8796036839, -0.4581531584,  0.1280334443, -0.1775462031],
        [-0.2590542734, -0.2355925292,  0.9366894364,  0.0082093179],
        [-0.3989834487, -0.8570833206, -0.3259147406,  0.8723040819]])


RuntimeError: [1;31m[Open3D ERROR] GLFW Error: X11: The DISPLAY environment variable is missing[0;m